In [20]:
import os
from pathlib import Path
from mne_bids import (read_raw_bids, BIDSPath,
                     get_entity_vals, get_datatypes,
                     make_report)
from mne_hfo import HilbertDetector
from mne_hfo.io import write_annotations
from mne_hfo.posthoc import compute_chs_hfo_rates, match_detected_annotations
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score, mutual_info_score 

In [2]:
reference = "monopolar"
l_freq = 80
h_freq = 250

fpaths = {
    "preresection": [],
    "intraresection": [],
    "postresection": []
}
# change this
bids_root = Path("C:/Users/patri/Johns Hopkins/Adam Li - sickkids")
subjects = get_entity_vals(bids_root, 'subject')
for subjectID in subjects:
    for sessionID in fpaths.keys():
        bids_paths = BIDSPath(subject=subjectID, session=sessionID,
                          datatype="ieeg",
                          suffix="ieeg",
                          extension=".vhdr", root=bids_root)
        fpaths[sessionID].extend(bids_paths.match())


In [6]:
kwargs = {
    'filter_band': (l_freq, h_freq),
    'threshold': 3,
    'band_method': 'log',
    'hfo_name': 'ripple',
    'n_jobs': 1
}
hil_detector = HilbertDetector(**kwargs)

In [22]:
def plot_and_save(df, fpath, outpath):
    ch_dict = compute_chs_hfo_rates(df, 'h', verbose=False)
    channels_path = fpath.copy().update(extension=".tsv", suffix="channels")
    channel_df = pd.read_csv(channels_path, sep='\t')
    channel_df = channel_df[channel_df["name"].isin(raw.ch_names)].reset_index()
    resected = channel_df[channel_df["description"] == "resected"]
    resected_chnames = list(resected["name"])
    chnames = ch_dict.keys()
    resected_idxs = [ind for ind, chname in enumerate(chnames) if chname in resected_chnames]
    xdat = ch_dict.keys()
    ydat = ch_dict.values()
    fig = plt.figure(figsize=(36, 10), dpi=80, facecolor='w', edgecolor='k')
    ax = fig.add_axes([0,0,1,1])
    barlist=ax.bar(xdat,ydat)
    plt.xticks(rotation = 90, fontsize=24)
    plt.yticks(fontsize=16)
    plt.xlabel('Channels', fontsize=24)
    plt.ylabel('HFOs per hour', fontsize=24)
    plt.title(f'HFOs for {fpath.subject} during {fpath.session}', fontsize=36)
    print(len(ax.get_xticklabels()))
    [ax.get_xticklabels()[idx].set_color("#DC143C") for idx in resected_idxs]
    [barlist[idx].set_color('#DC143C') for idx in resected_idxs]
    os.makedirs(outpath.parent, exist_ok=True)
    plt.savefig(str(outpath))
    # save the figure

In [7]:
for session, filepaths in fpaths.items():
    for fpath in filepaths:
        raw = read_raw_bids(fpath)
        raw = raw.drop_channels(raw.info['bads'])
        raw = raw.pick_types(ecog=True, seeg=True)
        hil_detector.fit(raw)
        df = hil_detector.df_
        outname = fpath.basename.replace(".vhdr", ".png")
        root = fpath.root
        subject = fpath.subject
        session = fpath.session
        figure_dir = root / "derivatives" / "figures" / "hfo"
        outdir  = figure_dir / f"sub-{subject}" / f"ses-{session}"
        outpath = outdir / outname
        outpath
        plot_and_save(df, fpath, outpath)


Extracting parameters from C:\Users\patri\Johns Hopkins\Adam Li - sickkids\sub-E1\ses-preresection\ieeg\sub-E1_ses-preresection_task-pre_acq-ecog_run-01_ieeg.vhdr...
Setting channel info structure...
Reading channel info from C:\Users\patri\Johns Hopkins\Adam Li - sickkids\sub-E1\ses-preresection\ieeg\sub-E1_ses-preresection_task-pre_acq-ecog_run-01_channels.tsv.
Reading in coordinate system frame other: None.
Reading electrode coords from C:\Users\patri\Johns Hopkins\Adam Li - sickkids\sub-E1\ses-preresection\ieeg\sub-E1_ses-preresection_acq-ecog_space-fs_electrodes.tsv.
The read in electrodes file is: 
 [('name', ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 

<ipython-input-7-04c0bfda86cd>:2: RuntimeWarning: Did not find any events.tsv associated with sub-E1_ses-preresection_task-pre_acq-ecog_run-01.

The search_str was "C:\Users\patri\Johns Hopkins\Adam Li - sickkids\sub-E1\**\sub-E1_ses-preresection*events.tsv"
  raw = read_raw_bids(fpath)
<ipython-input-7-04c0bfda86cd>:2: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50', 'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C58', 'C59', 'C60', 'C61', 'C62', 'C63', 'C64', 'IPD1', 'IPD2', 'IPD3', 'IPD4', 'IPD5', 'IPD6', 'IAD1', 'IAD2', 'IAD3', 'IAD4', 'IAD5', 'IAD6', 'SAD1', 'SAD2', 'SAD3', 'SAD4', 'SAD5', 'SAD6', 'SPD1', 'SPD2', 'SPD3

 30%|██▉       | 18/61 [00:00<00:00, 57.01HFO-first-phase/s]


 34%|███▎      | 33/98 [00:40<01:20,  1.25s/it]


 21%|██▏       | 13/61 [00:00<00:01, 30.85HFO-first-phase/s]


...

 72%|███████▏  | 71/98 [01:50<00:45,  1.69s/it]


 20%|█▉        | 12/61 [00:00<00:01, 25.79HFO-first-phase/s]


 36%|███▌      | 22/61 [00:00<00:00, 54.34HFO-first-phase/s]


100%|██████████| 98/98 [02:37<00:00,  1.60s/it]


NameError: name 'll_detector' is not defined